In [1]:
import json
import numpy as np
import pandas as pd
import uproot
import matplotlib.pyplot as plt
import mplhep as mh

from hist.hist import Hist
from hist.axis import Regular
from hist.axis import IntCategory
from hist.axis import StrCategory

from pathlib import Path

from typing import Optional, Union

from matplotlib.colors import Colormap, ListedColormap
from matplotlib.colors import LogNorm

In [2]:
from functools import cache
from functools import cached_property

@cache
def get_segment(ring: int, station: int, sector: int, subsector: int) -> int:
    """
    https://github.com/cms-sw/cmssw/blob/CMSSW_13_3_0_pre3/Geometry/RPCGeometry/src/RPCGeomServ.cc#L361-L368
    """
    nsub = 3 if ring == 1 and station > 1 else 6
    return subsector + nsub * (sector - 1)


@cache
def get_roll_name(region: int, ring: int, station: int, sector: int, layer: int,
             subsector: int, roll: int
) -> str:
    """
    https://github.com/cms-sw/cmssw/blob/CMSSW_13_3_0_pre3/Geometry/RPCGeometry/src/RPCGeomServ.cc#L11-L87
    """
    if region == 0:
        name = f'W{ring:+d}_RB{station}'

        if station <= 2:
            name += 'in' if layer == 1 else 'out'
        else:
            if sector == 4 and station == 4:
                name += ['--', '-', '+', '++'][subsector - 1]
            elif (station == 3) or (station == 4 and sector not in (4, 9, 11)):
                name += '-' if subsector == 1 else '+'
        name += f'_S{sector:0>2d}_'
        name += ['Backward', 'Middle', 'Forward'][roll - 1]
    else:
        segment = get_segment(ring, station, sector, subsector)
        name = f'RE{station * region:+d}_R{ring}_CH{segment:0>2d}_'
        name += ['A', 'B', 'C', 'D', 'E'][roll - 1]
    return name

In [3]:
def load_data(input_path: Path,
                       roll_blacklist_path: Optional[Path] = None,
)->dict:
    data = uproot.open(f"{str(input_path)}:tree").arrays(library='np')

    if roll_blacklist_path is None:
        roll_blacklist = set()
    else:
        with open(roll_blacklist_path) as stream:
            roll_blacklist = set(json.load(stream))

    fiducial_mask = data['is_fiducial']

    for key, values in data.items():
        data[key] = data[key][fiducial_mask]

    data['roll_name'] = np.array([get_roll_name(
                                                data['region'][idx], data['ring'][idx], data['station'][idx],
                                                data['sector'][idx], data['layer'][idx], data['subsector'][idx], data['roll'][idx]) 
                                                for idx in range(len(data['region']))])
                                            
    is_blacklist = np.vectorize(lambda item: item in roll_blacklist)
    blacklist_mask = is_blacklist(data['roll_name'])

    for key, values in data.items():
        data[key] = data[key][~blacklist_mask]

    return data

In [4]:
working_dir = Path('/users/eigen1907/Workspace/Workspace-RPC/240425-TnP_RPC24/TnP_Plotting')

data = load_data(
    input_path = working_dir / 'data' / 'Run2022.root',
    roll_blacklist_path = working_dir / 'blacklist' / 'roll-blacklist.json',
)

In [8]:
region = 'all'

if region == "all":
    is_region = np.vectorize(lambda item: type(item) is str)
elif region == "barrel":
    is_region = np.vectorize(lambda item: item.startswith('W'))
elif region == "endcap":
    is_region = np.vectorize(lambda item: item.startswith('RE'))

region_mask = is_region(data['roll_name'])

region_data = {}

for key, values in data.items():
    region_data[key] = data[key][region_mask]

runs = np.unique(region_data['run'])
run_effs = []

for run in runs:
    run_matched = region_data['is_matched'][region_data['run'] == run]
    run_eff = len(run_matched[run_matched]) / len(run_matched)
    run_effs.append(run_eff)

run_effs = np.array(run_effs)

print(runs)
print(run_effs)

[ True False  True  True False False False False  True  True  True  True
 False  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True]
31
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True]
25


In [ ]:
working_dir = Path('/users/eigen1907/Workspace/Workspace-RPC/240425-TnP_RPC24/TnP_Plotting')

mh.style.use(mh.styles.CMS)
fig, ax = plt.subplots(figsize=(16, 10))

label = "Work in Progress"
com = 13.6

mh.cms.label(ax=ax, data=True, label=label, com=com, year="RPC Efficiency - Endcap", fontsize=30)
ax.set_xlabel('Efficiency [%]', fontsize=24)
ax.set_ylabel('Number of Rolls', fontsize=24)
ax.set_xlim(70, 100)
ax.set_ylim(0, 450)
ax.plot(np.NaN, np.NaN, '-', color="None", label="           Mean(>70%)   %(<70%)")

ax = plot_eff_hist(
    input_path = working_dir / 'data/Run2022.root',
    roll_blacklist_path = working_dir / 'blacklist/roll-blacklist.json',
    region = "endcap",
    ax = ax,
    facecolor="#7CA1FF",
    edgecolor = "#000775",
    data_legend = "2022",
    hatch = '///'
)

ax = plot_eff_hist(
    input_path = working_dir / 'data/Run2023.root',
    roll_blacklist_path = working_dir / 'blacklist/roll-blacklist.json',
    region = "endcap",
    ax = ax,
    facecolor="#0714FF",
    edgecolor = "#000775",
    data_legend = "2023"
)

ax.legend(fontsize="26", loc='upper left')

output_dir = working_dir / 'plotting' / 'eff-hist'

if not output_dir.exists():
    output_dir.mkdir(parents=True)
    
fig.savefig(output_dir / "run3-endcap.png")